In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time
import io
import os
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
import os
import sys
import json
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

# import seaborn as sns
import plotly
import plotly.offline as pyoff
# import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go
init_notebook_mode(connected = True)
# %matplotlib inline

In [4]:
os.listdir("gdrive/My Drive/Keras_update/predictcuisine/")

['train.json', 'test.json']

### The following steps are to setup tensorboard on colab.

In [114]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip



Redirecting output to ‘wget-log.3’.
Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
http://c1bed704.ngrok.io


In [115]:

LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://c1bed704.ngrok.io


In [0]:
def load_data(dataset):
    """This function takes the path to the dataset as input and converts the json format to a pandas dataframe."""
    # Load Data
    with open(dataset, 'r') as t:
        data = json.load(t)
    
    # Creating a dictionary out of the data
    data_dict = {}
    for i in data:
        data_dict[i['id']] = i
    
    # Creating a Dataframe 
    df = pd.DataFrame.from_dict(data_dict).transpose()
    df.reindex(df['id'])
    return df

In [0]:
train_df = load_data('gdrive/My Drive/Keras_update/predictcuisine/train.json')

In [7]:
train_df.head()

,cuisine,id,ingredients
0,spanish,0,"[mussels, ground black pepper, garlic cloves, ..."
1,mexican,1,"[tomatoes, diced red onions, paprika, salt, co..."
2,french,2,"[chicken broth, truffles, pimentos, green pepp..."
3,chinese,3,"[fresh ginger, sesame oil, frozen peas, cooked..."
4,italian,4,"[orange peel, cookies, vanilla ice cream, gran..."


In [0]:
test_df = load_data('gdrive/My Drive/Keras_update/predictcuisine/test.json')

In [9]:
test_df.head()

,id,ingredients
5,5,"[mushrooms, chopped onion, tomato sauce, chees..."
7,7,"[minced garlic, brown rice, sour cream, chicke..."
11,11,"[lime juice, sesame oil, garlic cloves, fish s..."
12,12,"[sugar, vanilla extract, corn starch, coffee g..."
13,13,"[frozen pie crust, bourbon whiskey, powdered s..."


##### Number of dishes in the data

In [10]:
print ("Number of dishes in the dataset is {}.".format(train_df.shape[0]))

Number of dishes in the dataset is 39774.


##### Number of cuisines

In [11]:
len(train_df.cuisine.unique())

20

##### Types of cuisines

In [12]:
train_df.cuisine.unique()

array(['spanish', 'mexican', 'french', 'chinese', 'italian',
       'southern_us', 'indian', 'thai', 'cajun_creole', 'jamaican',
       'japanese', 'greek', 'russian', 'irish', 'moroccan', 'korean',
       'filipino', 'vietnamese', 'british', 'brazilian'], dtype=object)

#### EDA

##### Which cuisine is most common?

###### Creating function to generate plotly layout

In [0]:
def generate_layout_bar(col_name):
    layout_bar = go.Layout(
        autosize=False, # auto size the graph? use False if you are specifying the height and width
        width=800, # height of the figure in pixels
        height=600, # height of the figure in pixels
        title = "Distribution of {} column".format(col_name), # title of the figure
        # more granular control on the title font 
        titlefont=dict( 
            family='Courier New, monospace', # font family
            size=14, # size of the font
            color='black' # color of the font
        ),
        # granular control on the axes objects 
        xaxis=dict( 
        tickfont=dict(
            family='Courier New, monospace', # font family
            size=14, # size of ticks displayed on the x axis
            color='black'  # color of the font
            )
        ),
        yaxis=dict(
#         range=[0,100],
            title='Percentage',
            titlefont=dict(
                size=14,
                color='black'
            ),
        tickfont=dict(
            family='Courier New, monospace', # font family
            size=14, # size of ticks displayed on the y axis
            color='black' # color of the font
            )
        ),
        font = dict(
            family='Courier New, monospace', # font family
            color = "white",# color of the font
            size = 12 # size of the font displayed on the bar
                )  
        )
    return layout_bar

In [0]:
def plot_bar(df_name, col_name):
    # create a table with value counts
    temp = df_natenme[col_name].value_counts()
    # creating a Bar chart object of plotly
    data = [go.Bar(
            x=temp.index.astype(str), # x axis values
            y=np.round(temp.values.astype(float)/temp.values.sum(),4)*100, # y axis values
            text = ['{}%'.format(i) for i in np.round(temp.values.astype(float)/temp.values.sum(),4)*100],
        # text to be displayed on the bar, we are doing this to display the '%' symbol along with the number on the bar
            textposition = 'auto', # specify at which position on the bar the text should appear
        marker = dict(color = '#0047AB'),)] # change color of the bar
    # color used here Cobalt Blue
     
    layout_bar = generate_layout_bar(col_name=col_name)

    fig = go.Figure(data=data, layout=layout_bar)
    return iplot(fig)

In [15]:
plot_bar(train_df,'cuisine')

From the above graph we infer that 
    * The most common cuisine is Italian
    * There is a Class imbalance

### On an average , which cuisine uses more ingredients

In [0]:
train_df['ingredients_count'] = train_df.ingredients.apply(lambda x : len(x))

In [17]:
train_df[train_df.cuisine=='brazilian'].ingredients_count.mean()

9.5203426124197

In [0]:
cuisine_avg_ing = train_df.groupby(['cuisine']).agg({'ingredients_count':np.mean}).reset_index().sort_values('ingredients_count',ascending = False)

In [19]:
cuisine_avg_ing.head()

,cuisine,ingredients_count
14,moroccan,12.909866
7,indian,12.705961
19,vietnamese,12.675152
2,cajun_creole,12.617076
18,thai,12.545809


Moroccan uses more ingredients on an average, closely follwed by indian!

In [20]:
cuisine_avg_ing.tail(2)

,cuisine,ingredients_count
0,brazilian,9.520343
8,irish,9.299850


Irish and brazilian dishes use the least ingredients

## Model building

In [21]:
!pip install nltk

In [0]:
import sys

In [0]:
# sys.path.append("c:\\users\\chait\\anaconda3\\lib\\site-packages")

In [0]:
from nltk.probability import FreqDist

##### Find the most common ingredients

In [0]:
def find_common_ingredient(df_name, cuisine_name):
    ing_list = [ing for i in df_name.ingredients[df_name.cuisine == cuisine_name] for ing in i ]
    return FreqDist(ing_list)

In [26]:
find_common_ingredient(train_df, 'indian')

FreqDist({'curry powder': 403,
          'fat free reduced sodium chicken broth': 2,
          'garlic cloves': 446,
          'fresh ginger': 379,
          'green peas': 71,
          'onions': 1195,
          'fresh cilantro': 106,
          'diced tomatoes': 123,
          'cooked shrimp': 3,
          'red potato': 17,
          'vegetable oil': 593,
          'salt': 1934,
          'curry leaves': 208,
          'green chilies': 484,
          'dal': 13,
          'red chili powder': 171,
          'ginger': 526,
          'garam masala': 862,
          'cashew nuts': 126,
          'cooking oil': 56,
          'cumin seed': 697,
          'ground turmeric': 728,
          'tomatoes': 521,
          'corn flour': 15,
          'bottle gourd': 4,
          'purple onion': 170,
          'ground cumin': 683,
          'paprika': 189,
          'greek yogurt': 89,
          'tumeric': 492,
          'lemon': 134,
          'chili powder': 485,
          'chicken thighs': 53,
      

###### A particular cuisine might use a set of ingredients, Let us see the most common ingredients

In [0]:
ing_freq_dict = {}
for i in train_df.cuisine.unique():
    ing_freq = find_common_ingredient(train_df, i)
    ing_freq_dict[i] = ing_freq

In [28]:
ing_freq_dict

{'brazilian': FreqDist({'1% low-fat milk': 1,
           '2% reduced-fat milk': 1,
           'Hawaiian salt': 1,
           'Italian parsley leaves': 2,
           'Italian seasoned breadcrumbs': 1,
           'Jell-O Gelatin Dessert': 1,
           'Sriracha': 1,
           'Sugar in the Raw': 1,
           'Tabasco Pepper Sauce': 4,
           'acai juice': 1,
           'active dry yeast': 3,
           'adobo style seasoning': 1,
           'agave nectar': 4,
           'alcohol': 1,
           'all purpose unbleached flour': 2,
           'all-purpose flour': 18,
           'allspice berries': 1,
           'almond butter': 3,
           'almond milk': 9,
           'amber': 1,
           'ancho powder': 1,
           'andouille sausage': 1,
           'anise seed': 1,
           'annatto': 1,
           'apple cider vinegar': 1,
           'apple juice': 1,
           'apple pie spice': 1,
           'apples': 5,
           'apricots': 1,
           'arrow root': 1,
           '

From the above output, we see that salt is very common among many cuisines.

For model building, I will follow the approach of using embeddings to find the dishes that are similar to each other.
It will be interesting to see if using only the embeddings we can calssify the cuisines

###### list all the ingredients

In [0]:
ing_set = set([ing for i in train_df.ingredients for ing in i])

###### We have to find the number of unique ingredients in the whole dataset.

In [30]:
input_dim= len(ing_set)
input_dim

6714

###### We have to find the dish with max number of ingredients

In [31]:
input_length = np.max([len(i) for i in train_df.ingredients])
input_length

65

In [0]:
ing_mapping_dict = dict(enumerate(ing_set,1))
ing_reverse_mapping_dict = {i[1]:i[0] for i in ing_mapping_dict.items()}

###### Map the integers to the words in data

In [33]:
train_df.head()

,cuisine,id,ingredients,ingredients_count
0,spanish,0,"[mussels, ground black pepper, garlic cloves, ...",11
1,mexican,1,"[tomatoes, diced red onions, paprika, salt, co...",18
2,french,2,"[chicken broth, truffles, pimentos, green pepp...",18
3,chinese,3,"[fresh ginger, sesame oil, frozen peas, cooked...",11
4,italian,4,"[orange peel, cookies, vanilla ice cream, gran...",4


In [0]:
def return_mapped_int_ing(ing):
    try:
        return ing_reverse_mapping_dict[ing]
    except:
        pass

In [0]:
train_df['mapped_ing'] = train_df.ingredients.apply(lambda x : [return_mapped_int_ing(i) for i in x])

In [36]:
train_df.head(10)

,cuisine,id,ingredients,ingredients_count,mapped_ing
0,spanish,0,"[mussels, ground black pepper, garlic cloves, ...",11,"[1121, 3686, 1618, 4700, 6356, 727, 2488, 1896..."
1,mexican,1,"[tomatoes, diced red onions, paprika, salt, co...",18,"[3223, 2580, 960, 3178, 5339, 371, 734, 3803, ..."
2,french,2,"[chicken broth, truffles, pimentos, green pepp...",18,"[6592, 4627, 886, 5862, 1500, 4490, 5978, 6654..."
3,chinese,3,"[fresh ginger, sesame oil, frozen peas, cooked...",11,"[1284, 575, 6332, 5279, 495, 2014, 5529, 2235,..."
4,italian,4,"[orange peel, cookies, vanilla ice cream, gran...",4,"[4115, 3366, 4964, 3975]"
6,chinese,6,"[olive oil, sesame oil, soy sauce, flowering g...",4,"[6356, 575, 4007, 1120]"
8,french,8,"[whole milk, fine sea salt, vanilla sugar, lem...",9,"[4056, 1262, 6571, 6260, 1162, 782, 1587, 5529..."
9,southern_us,9,"[sweet potatoes, all-purpose flour, pepper, ap...",11,"[3848, 1587, 2303, 2297, 3963, 682, 3266, 4049..."
10,chinese,10,"[black pepper, crushed red pepper flakes, top ...",10,"[3855, 6333, 2647, 3543, 4304, 495, 5681, 4007..."
14,italian,14,"[olive oil, balsamic vinegar, toasted pine nut...",12,"[6356, 4996, 1430, 1952, 5181, 349, 3871, 2706..."


#### Train and Test split for modelling

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X = train_df.mapped_ing.values
y = train_df.cuisine.values

In [0]:
from tensorflow.keras.utils import to_categorical

In [0]:
n_classes= len(train_df.cuisine.unique())

In [41]:
n_classes

20

In [0]:
classes = dict(enumerate(train_df.cuisine.unique()))

In [43]:
classes

{0: 'spanish',
 1: 'mexican',
 2: 'french',
 3: 'chinese',
 4: 'italian',
 5: 'southern_us',
 6: 'indian',
 7: 'thai',
 8: 'cajun_creole',
 9: 'jamaican',
 10: 'japanese',
 11: 'greek',
 12: 'russian',
 13: 'irish',
 14: 'moroccan',
 15: 'korean',
 16: 'filipino',
 17: 'vietnamese',
 18: 'british',
 19: 'brazilian'}

In [0]:
classes_reverse_mapping = {i[1]:i[0] for i in classes.items()}

In [0]:
cuisine_mappings = train_df.cuisine.replace(classes_reverse_mapping)

In [0]:
train_df['cuisine_mappings'] = cuisine_mappings

##### Let us validate if we did it right

In [47]:
classes

{0: 'spanish',
 1: 'mexican',
 2: 'french',
 3: 'chinese',
 4: 'italian',
 5: 'southern_us',
 6: 'indian',
 7: 'thai',
 8: 'cajun_creole',
 9: 'jamaican',
 10: 'japanese',
 11: 'greek',
 12: 'russian',
 13: 'irish',
 14: 'moroccan',
 15: 'korean',
 16: 'filipino',
 17: 'vietnamese',
 18: 'british',
 19: 'brazilian'}

In [48]:
train_df.head(10)

,cuisine,id,ingredients,ingredients_count,mapped_ing,cuisine_mappings
0,spanish,0,"[mussels, ground black pepper, garlic cloves, ...",11,"[1121, 3686, 1618, 4700, 6356, 727, 2488, 1896...",0
1,mexican,1,"[tomatoes, diced red onions, paprika, salt, co...",18,"[3223, 2580, 960, 3178, 5339, 371, 734, 3803, ...",1
2,french,2,"[chicken broth, truffles, pimentos, green pepp...",18,"[6592, 4627, 886, 5862, 1500, 4490, 5978, 6654...",2
3,chinese,3,"[fresh ginger, sesame oil, frozen peas, cooked...",11,"[1284, 575, 6332, 5279, 495, 2014, 5529, 2235,...",3
4,italian,4,"[orange peel, cookies, vanilla ice cream, gran...",4,"[4115, 3366, 4964, 3975]",4
6,chinese,6,"[olive oil, sesame oil, soy sauce, flowering g...",4,"[6356, 575, 4007, 1120]",3
8,french,8,"[whole milk, fine sea salt, vanilla sugar, lem...",9,"[4056, 1262, 6571, 6260, 1162, 782, 1587, 5529...",2
9,southern_us,9,"[sweet potatoes, all-purpose flour, pepper, ap...",11,"[3848, 1587, 2303, 2297, 3963, 682, 3266, 4049...",5
10,chinese,10,"[black pepper, crushed red pepper flakes, top ...",10,"[3855, 6333, 2647, 3543, 4304, 495, 5681, 4007...",3
14,italian,14,"[olive oil, balsamic vinegar, toasted pine nut...",12,"[6356, 4996, 1430, 1952, 5181, 349, 3871, 2706...",4


- All the mapping are correct

In [0]:
y = to_categorical(train_df.cuisine_mappings,num_classes=n_classes)

In [50]:
y.shape

(39774, 20)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [52]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((26648,), (13126,), (26648, 20), (13126, 20))

In [0]:
from tensorflow.keras.layers import Dense,Dropout, CuDNNLSTM,LSTM, Embedding, SimpleRNN, Flatten, Conv1D # Import layers from Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

##### Defining recall

In [0]:
import tensorflow.keras.backend as K # This 'K' can be used to create user defined functions in keras

# Define a custom function in keras to compute recall.
# Arguments:
# y_true - Actual labels
# y_pred - Predicted labels
def recall(y_true, y_pred):
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    PP = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = TP / (PP + K.epsilon())
    return recall

####  our network cannot deal with different length sequences so we pad the sequences with zeros and make all the sequences equal in length
We also have to embed out ingredients with numeric values, so that out network can be fit

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [57]:
X_train[0]

[948, 2235, 935, 4695, 4736, 5323, 436, 3178, 2114, 700, 1255, 4194, 83]

In [0]:
def pad_seq(seq, output_length):
    tmp = np.zeros(output_length)
    tmp[-len(seq):] = seq
    return tmp

###### Padding sequences

In [0]:
X_train = np.array([pad_seq(i, input_length) for i in X_train])

In [0]:
X_test = np.array([pad_seq(i, input_length) for i in X_test])

In [61]:
X_train.shape,X_test.shape

((26648, 65), (13126, 65))

### LSTM Model

In [0]:
# Building an LSTM model
model = Sequential() # Call Sequential to initialize a network
model.add(Embedding(input_dim=input_dim, 
                    input_length=input_length, 
                    output_dim=32)) # Add an embedding layer which represents each unique token as a vector
# model.add(Flatten())
model.add(CuDNNLSTM(128))# Add an LSTM layer
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))

model.add(Dense(8, activation='relu')) 
model.add(Dense(n_classes, activation='softmax')) # Add an ouput layer.

In [0]:
tensorboard = TensorBoard(log_dir="./log/{}".format("LSTM_Model"))

In [0]:
callbacks = [EarlyStopping(monitor='val_loss', min_delta= 0.005, patience=10),
            EarlyStopping(monitor='val_acc', min_delta= 0.005, patience=10),
            EarlyStopping(monitor='val_recall', min_delta= 0.005, patience=10),tensorboard]

In [119]:
# Mention the optimizer, Loss function and metrics to be computed
model.compile(optimizer='adam',  # 'Adam' is a variant of gradient descent technique
              loss='categorical_crossentropy', # categorical_crossentropy for multi-class classification
              metrics=['accuracy', recall]) # These metrics are computed and printed for evaluating
model.fit(X_train, y_train, 
                   callbacks=callbacks, # Early stopping
                   epochs=200, validation_split=0.25)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning:

Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.



Train on 19986 samples, validate on 6662 samples
Epoch 1/200
19986/19986 [==============================] - 17s 844us/step - loss: 0.1671 - acc: 0.9483 - recall: 0.9365 - val_loss: 2.3325 - val_acc: 0.6764 - val_recall: 0.6663
Epoch 2/200
19986/19986 [==============================] - 16s 801us/step - loss: 0.1466 - acc: 0.9542 - recall: 0.9446 - val_loss: 2.4796 - val_acc: 0.6789 - val_recall: 0.6683
Epoch 3/200
19986/19986 [==============================] - 16s 806us/step - loss: 0.1463 - acc: 0.9534 - recall: 0.9427 - val_loss: 2.4104 - val_acc: 0.6816 - val_recall: 0.6699
Epoch 4/200
19986/19986 [==============================] - 16s 812us/step - loss: 0.1368 - acc: 0.9558 - recall: 0.9475 - val_loss: 2.4343 - val_acc: 0.6797 - val_recall: 0.6687
Epoch 5/200
19986/19986 [==============================] - 16s 816us/step - loss: 0.1336 - acc: 0.9567 - recall: 0.9479 - val_loss: 2.5247 - val_acc: 0.6768 - val_recall: 0.6666
Epoch 6/200
19986/19986 [==============================] - 16

### Prediction and evaluation on test data
1. Check the network output on test data. What do these values represent?
2. Predict the class labels on test data
2. Evaluate the model on test data

Hint: Check model.predict, model.predict_classes, model.evaluate in keras

In [74]:
model.metrics_names

['loss', 'acc', 'recall']

In [75]:
model.evaluate(X_test, y_test)

13126/13126 [==============================] - 3s 216us/step


[1.2682397922428028, 0.6812433338321492, 0.6322565899650153]

In [0]:
import h5py

In [120]:
model_json = model.to_json()
with open("gdrive/My Drive/Keras_update/predictcuisine/model_LSTM.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("gdrive/My Drive/Keras_update/predictcuisine/model.h5")
print("Saved model to disk")

Saved model to disk


In [121]:
from tensorflow.keras.models import model_from_json

# load json and create model
json_file = open('gdrive/My Drive/Keras_update/predictcuisine/model_LSTM.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model_LSTM = model_from_json(loaded_model_json)
# load weights into new model
loaded_model_LSTM.load_weights("gdrive/My Drive/Keras_update/predictcuisine/model.h5")
print("Loaded model from disk")

loaded_model_LSTM.compile(optimizer='adam',  # 'Adam' is a variant of gradient descent technique
              loss='categorical_crossentropy', # categorical_crossentropy for multi-class classification
              metrics=['accuracy', recall]) # These metrics are computed and printed for evaluating

Loaded model from disk


In [122]:
loaded_model_LSTM.evaluate(X_test, y_test)

13126/13126 [==============================] - 4s 305us/step


[2.7560737055205418, 0.6683681243333841, 0.6617400579003504]

#### Creating submission file

In [0]:
test_df['mapped_ing'] = test_df.ingredients.apply(lambda x : [return_mapped_int_ing(i) for i in x])

test_x = np.array([pad_seq(i, input_length) for i in test_df.mapped_ing.values])

# test_x

preds = loaded_model_LSTM.predict_classes(test_x)

# preds

# np.unique(preds)

preds = [classes[i] for i in preds]

sub = pd.DataFrame(list(zip(test_df.id.values,preds)),columns= ['id', 'cuisine'])
sub.to_csv('gdrive/My Drive/Keras_update/predictcuisine/LSTM_sub.csv')
# sub

#### MLP

In [125]:
NAME  = "MLPModel"
# Building an MLP model
model = Sequential() # Call Sequential to initialize a network
model.add(Embedding(input_dim=input_dim, 
                    input_length=input_length, 
                    output_dim=32)) # Add an embedding layer which represents each unique token as a vector
model.add(Flatten())
model.add(Dense(512))# Add an LSTM layer
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu')) 

model.add(Dense(n_classes, activation='softmax')) # Add an ouput layer.

tensorboard = TensorBoard(log_dir="./log/{}".format(NAME))

callbacks = [EarlyStopping(monitor='val_loss', min_delta= 0.005, patience=10),
            EarlyStopping(monitor='val_acc', min_delta= 0.005, patience=10),
            EarlyStopping(monitor='val_recall', min_delta= 0.005, patience=10), tensorboard]
# Mention the optimizer, Loss function and metrics to be computed



model.compile(optimizer='adam',  # 'Adam' is a variant of gradient descent technique
              loss='categorical_crossentropy', # categorical_crossentropy for multi-class classification
              metrics=['accuracy', recall]) # These metrics are computed and printed for evaluating
model.fit(X_train, y_train, 
                   callbacks=callbacks, # Early stopping
                   epochs=1500, validation_split=0.25)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning:

Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.



Train on 19986 samples, validate on 6662 samples
Epoch 1/1500
19986/19986 [==============================] - 9s 433us/step - loss: 2.1604 - acc: 0.3466 - recall: 0.1827 - val_loss: 1.5328 - val_acc: 0.5306 - val_recall: 0.3852
Epoch 2/1500
19986/19986 [==============================] - 7s 355us/step - loss: 1.2886 - acc: 0.6214 - recall: 0.4934 - val_loss: 1.2475 - val_acc: 0.6366 - val_recall: 0.5305
Epoch 3/1500
19986/19986 [==============================] - 7s 354us/step - loss: 0.9892 - acc: 0.7061 - recall: 0.6086 - val_loss: 1.2351 - val_acc: 0.6518 - val_recall: 0.5663
Epoch 4/1500
19986/19986 [==============================] - 7s 354us/step - loss: 0.7814 - acc: 0.7665 - recall: 0.6876 - val_loss: 1.2687 - val_acc: 0.6509 - val_recall: 0.5716
Epoch 5/1500
19986/19986 [==============================] - 7s 351us/step - loss: 0.6122 - acc: 0.8134 - recall: 0.7520 - val_loss: 1.3945 - val_acc: 0.6506 - val_recall: 0.5959
Epoch 6/1500
19986/19986 [==============================] - 7

In [81]:
model.metrics_names

['loss', 'acc', 'recall']

In [82]:
model.evaluate(X_test, y_test)

13126/13126 [==============================] - 1s 79us/step


[2.3697767292655825, 0.6187719031021796, 0.600487581907604]

In [0]:
def save_model(model, name):
    model_json = model.to_json()
    with open("gdrive/My Drive/Keras_update/predictcuisine/{}.json".format(name), "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("gdrive/My Drive/Keras_update/predictcuisine/{}_weights.h5".format(name))
    print("Saved model to disk : {}".format(name))

In [90]:
save_model(model=model, name="Dense_With_Dropout")

Saved model to disk : Dense_With_Dropout


In [0]:
def load_model(name):
    from tensorflow.keras.models import model_from_json
    # load json and create model
    json_file = open("gdrive/My Drive/Keras_update/predictcuisine/{}.json".format(name), 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("gdrive/My Drive/Keras_update/predictcuisine/{}_weights.h5".format(name))
    print("Loaded model from disk : {}".format(name))
    return loaded_model

In [92]:
loaded_model_Dense_With_Dropout = load_model("Dense_With_Dropout")

Loaded model from disk : Dense_With_Dropout


In [95]:
loaded_model_Dense_With_Dropout.compile(optimizer='adam',  # 'Adam' is a variant of gradient descent technique
              loss='categorical_crossentropy', # categorical_crossentropy for multi-class classification
              metrics=['accuracy', recall]) # These metrics are computed and printed for evaluating

loaded_model_Dense_With_Dropout.evaluate(X_test, y_test)

13126/13126 [==============================] - 1s 99us/step


[2.3697767292655825, 0.6187719031021796, 0.600487581907604]

In [0]:
test_df['mapped_ing'] = test_df.ingredients.apply(lambda x : [return_mapped_int_ing(i) for i in x])

test_x = np.array([pad_seq(i, input_length) for i in test_df.mapped_ing.values])

# test_x

preds = loaded_model_Dense_With_Dropout.predict_classes(test_x)

# preds

# np.unique(preds)

preds = [classes[i] for i in preds]

sub = pd.DataFrame(list(zip(test_df.id.values,preds)),columns= ['id', 'cuisine'])
sub.to_csv('gdrive/My Drive/Keras_update/predictcuisine/DenseDropout_sub.csv',index=False)
# sub

#### Recall is the metric on Kaggle

In [126]:
# Building an LSTM model
model = Sequential() # Call Sequential to initialize a network
model.add(Embedding(input_dim=input_dim, 
                    input_length=input_length, 
                    output_dim=32)) # Add an embedding layer which represents each unique token as a vector
# model.add(Flatten())
model.add(CuDNNLSTM(128))

model.add(Dense(512))# Add an LSTM layer
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu')) 

model.add(Dense(n_classes, activation='softmax')) # Add an ouput layer. Since classification, 3 nodes for 3 classes.
callbacks = [EarlyStopping(monitor='val_loss', min_delta= 0.005, patience=10),
            EarlyStopping(monitor='val_acc', min_delta= 0.005, patience=10),
            EarlyStopping(monitor='val_recall', min_delta= 0.005, patience=10)]
# Mention the optimizer, Loss function and metrics to be computed
model.compile(optimizer='adam',  # 'Adam' is a variant of gradient descent technique
              loss='categorical_crossentropy', # categorical_crossentropy for multi-class classification
              metrics=['accuracy', recall]) # These metrics are computed and printed for evaluating
model.fit(X_train, y_train, 
                   callbacks=callbacks, # Early stopping
                   epochs=1500, validation_split=0.25)
model.evaluate(X_test, y_test)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning:

Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.



Train on 19986 samples, validate on 6662 samples
Epoch 1/1500
19986/19986 [==============================] - 18s 895us/step - loss: 2.1824 - acc: 0.2802 - recall: 0.1059 - val_loss: 1.7705 - val_acc: 0.4314 - val_recall: 0.2912
Epoch 2/1500
19986/19986 [==============================] - 16s 794us/step - loss: 1.5335 - acc: 0.5244 - recall: 0.3642 - val_loss: 1.3736 - val_acc: 0.5878 - val_recall: 0.4191
Epoch 3/1500
19986/19986 [==============================] - 16s 792us/step - loss: 1.2015 - acc: 0.6292 - recall: 0.5069 - val_loss: 1.2187 - val_acc: 0.6274 - val_recall: 0.5242
Epoch 4/1500
19986/19986 [==============================] - 16s 797us/step - loss: 1.0264 - acc: 0.6759 - recall: 0.5808 - val_loss: 1.1732 - val_acc: 0.6444 - val_recall: 0.5339
Epoch 5/1500
19986/19986 [==============================] - 16s 793us/step - loss: 0.9061 - acc: 0.7088 - recall: 0.6169 - val_loss: 1.1591 - val_acc: 0.6531 - val_recall: 0.5522
Epoch 6/1500
19986/19986 [==============================

[1.2462762720824996, 0.6919853725339624, 0.6513027578760316]

In [0]:
test_df['mapped_ing'] = test_df.ingredients.apply(lambda x : [return_mapped_int_ing(i) for i in x])

test_x = np.array([pad_seq(i, input_length) for i in test_df.mapped_ing.values])

preds = model.predict_classes(test_x)
preds = [classes[i] for i in preds]

sub = pd.DataFrame(list(zip(test_df.id.values,preds)),columns= ['id', 'cuisine'])
sub.to_csv('gdrive/My Drive/Keras_update/predictcuisine/LSTMDenseDropout_sub.csv',index=False)

In [0]:
model = Sequential() # Call Sequential to initialize a network
model.add(Embedding(input_dim=input_dim, 
                    input_length=input_length, 
                    output_dim=32)) # Add an embedding layer which represents each unique token as a vector
# model.add(Flatten())
model.add(CuDNNLSTM(128))

model.add(Dense(512))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu')) 

model.add(Dense(n_classes, activation='softmax')) # Add an ouput layer. Since classification, 3 nodes for 3 classes.
callbacks = [EarlyStopping(monitor='val_loss', min_delta= 0.005, patience=10),
            EarlyStopping(monitor='val_acc', min_delta= 0.005, patience=10),
            EarlyStopping(monitor='val_recall', min_delta= 0.005, patience=10)]
# Mention the optimizer, Loss function and metrics to be computed
model.compile(optimizer='adam',  # 'Adam' is a variant of gradient descent technique
              loss='categorical_crossentropy', # categorical_crossentropy for multi-class classification
              metrics=['accuracy', recall]) # These metrics are computed and printed for evaluating
model.fit(X_train, y_train, 
                   callbacks=callbacks, # Early stopping
                   epochs=1500, validation_split=0.25)
model.evaluate(X_test, y_test)

test_df['mapped_ing'] = test_df.ingredients.apply(lambda x : [return_mapped_int_ing(i) for i in x])

test_x = np.array([pad_seq(i, input_length) for i in test_df.mapped_ing.values])

# test_x

preds = model.predict_classes(test_x)

# preds

# np.unique(preds)

preds = [classes[i] for i in preds]

sub = pd.DataFrame(list(zip(test_df.id.values,preds)),columns= ['id', 'cuisine'])
sub.to_csv('LSTMDenseDropout_sub.csv',index=False)
# sub

In [0]:
def save_model(model, name):
    model_json = model.to_json()
    with open("gdrive/My Drive/Keras_update/predictcuisine/{}.json".format(name), "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("gdrive/My Drive/Keras_update/predictcuisine/{}_weights.h5".format(name))
    print("Saved model to disk : {}".format(name))

In [0]:
def load_model(name):
    from tensorflow.keras.models import model_from_json
    # load json and create model
    json_file = open("gdrive/My Drive/Keras_update/predictcuisine/{}.json".format(name), 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("gdrive/My Drive/Keras_update/predictcuisine/{}_weights.h5".format(name))
    print("Loaded model from disk : {}".format(name))
    return loaded_model

In [0]:
test_df['mapped_ing'] = test_df.ingredients.apply(lambda x : [return_mapped_int_ing(i) for i in x])

test_x = np.array([pad_seq(i, input_length) for i in test_df.mapped_ing.values])

In [136]:

dense_layers = [0, 1, 2, 3, 4]
layer_sizes = [32, 64, 128]
embedding_sizes = [32, 64]


for dense_layer in dense_layers:
  for layer_size in layer_sizes:
    for emb_size in embedding_sizes:
      NAME = "{}-emb_size-{}-nodes-{}-dense-{}".format(emb_size, layer_size, dense_layer, int(time.time()))
      print(NAME)

      model = Sequential()
      model.add(Embedding(input_dim=input_dim, 
                    input_length=input_length, 
                    output_dim=emb_size)) # Add an embedding layer which represents each unique token as a vector
      # model.add(Flatten())
      model.add(CuDNNLSTM(128))

      for _ in range(dense_layer):
        model.add(Dense(layer_size))
        model.add(Activation('relu'))

      model.add(Dense(n_classes))
      model.add(Activation('softmax'))

      tensorboard = TensorBoard(log_dir="gdrive/My Drive/Keras_update/predictcuisine/log/{}".format(NAME))

      model.compile(optimizer='adam',  # 'Adam' is a variant of gradient descent technique
              loss='categorical_crossentropy', # categorical_crossentropy for multi-class classification
              metrics=['accuracy', recall]) # These metrics are computed and printed for evaluating
      
      callbacks = [EarlyStopping(monitor='val_loss', min_delta= 0.005, patience=10),
            EarlyStopping(monitor='val_acc', min_delta= 0.005, patience=10),
            EarlyStopping(monitor='val_recall', min_delta= 0.005, patience=10), tensorboard]
      
      print(model.summary())
      model.fit(X_train, y_train, 
                         callbacks=callbacks, # Early stopping
                         epochs=1500, validation_split=0.25)
      
      save_model(model=model, name=NAME)
      
      print(model.evaluate(X_test, y_test))
    
      preds = model.predict_classes(test_x)

      preds = [classes[i] for i in preds]

      sub = pd.DataFrame(list(zip(test_df.id.values,preds)),columns= ['id', 'cuisine'])
      sub.to_csv('gdrive/My Drive/Keras_update/predictcuisine/{}.csv'.format(NAME),index=False)
      # sub


32-emb_size-32-nodes-0-dense-1537956292
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_41 (Embedding)     (None, 65, 32)            214848    
_________________________________________________________________
cu_dnnlstm_37 (CuDNNLSTM)    (None, 128)               82944     
_________________________________________________________________
dense_127 (Dense)            (None, 20)                2580      
_________________________________________________________________
activation_94 (Activation)   (None, 20)                0         
Total params: 300,372
Trainable params: 300,372
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning:

Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.



Train on 19986 samples, validate on 6662 samples
Epoch 1/1500
19986/19986 [==============================] - 16s 804us/step - loss: 1.7236 - acc: 0.4867 - recall: 0.3028 - val_loss: 1.2228 - val_acc: 0.6324 - val_recall: 0.4548
Epoch 2/1500
19986/19986 [==============================] - 15s 762us/step - loss: 1.0232 - acc: 0.6851 - recall: 0.5727 - val_loss: 1.0431 - val_acc: 0.6740 - val_recall: 0.5686
Epoch 3/1500
19986/19986 [==============================] - 15s 751us/step - loss: 0.8348 - acc: 0.7420 - recall: 0.6462 - val_loss: 0.9490 - val_acc: 0.7101 - val_recall: 0.6195
Epoch 4/1500
19986/19986 [==============================] - 15s 749us/step - loss: 0.6998 - acc: 0.7862 - recall: 0.7078 - val_loss: 0.9402 - val_acc: 0.7150 - val_recall: 0.6455
Epoch 5/1500
19986/19986 [==============================] - 15s 748us/step - loss: 0.6108 - acc: 0.8159 - recall: 0.7497 - val_loss: 0.9314 - val_acc: 0.7208 - val_recall: 0.6540
Epoch 6/1500
19986/19986 [==============================